In [ ]:
!pip install kagglehub torch torchvision pandas matplotlib scikit-learn

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import pandas as pd
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)



folder_path = '/content/drive/MyDrive/ML Project Team/dataset'
os.chdir(folder_path)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!ls

best_fashion_model.pth	images	myntradataset  styles.csv


In [ ]:
APPAREL_ARTICLES = [
    'Shirts', 'Jeans', 'Track Pants', 'Tshirts', 'Tops', 'Bra', 'Sweatshirts',
    'Kurtas', 'Waistcoat', 'Shorts', 'Briefs', 'Sarees', 'Innerwear Vests',
    'Rain Jacket', 'Dresses', 'Night suits', 'Skirts', 'Blazers', 'Kurta Sets',
    'Shrug', 'Trousers', 'Camisoles', 'Boxers', 'Dupatta', 'Capris', 'Bath Robe',
    'Tunics', 'Jackets', 'Trunk', 'Lounge Pants', 'Sweaters', 'Tracksuits',
    'Swimwear', 'Nightdress', 'Baby Dolls', 'Leggings', 'Kurtis', 'Jumpsuit',
    'Suspenders', 'Robe', 'Salwar and Dupatta', 'Patiala', 'Stockings',
    'Tights', 'Churidar', 'Lounge Tshirts', 'Lounge Shorts', 'Shapewear',
    'Nehru Jackets', 'Salwar', 'Jeggings', 'Rompers', 'Booties',
    'Lehenga Choli', 'Clothing Set', 'Belts', 'Rain Trousers', 'Suits'
]

FOOTWEAR_ARTICLES = [
    'Casual Shoes', 'Flip Flops', 'Sandals', 'Formal Shoes',
    'Flats', 'Sports Shoes', 'Heels', 'Sports Sandals'
]

In [ ]:
class FashionDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, labels_df, transform=None):
        self.img_dir = img_dir
        self.labels_df = labels_df
        self.transform = transform

        # Get all unique article types from the dataset
        self.classes = sorted(labels_df['articleType'].unique())
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_id = self.labels_df.iloc[idx]['id']
        img_path = os.path.join('images', f'{img_id}.jpg')

        image = Image.open(img_path).convert('RGB')
        label = self.class_to_idx[self.labels_df.iloc[idx]['articleType']]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
class FashionResNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # use resnet18 from cv
        self.model = resnet18(pretrained=True)

        # want most of the layers frozen, unfreeze final layer
        for param in self.model.parameters():
            param.requires_grad = False

        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)

        # Print parameter stats
        total_params = sum(p.numel() for p in self.model.parameters())
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print(f'\nTotal parameters: {total_params:,}')
        print(f'Trainable parameters: {trainable_params:,}')
        print(f'Frozen parameters: {total_params - trainable_params:,}')

        self.loss_criterion = nn.CrossEntropyLoss()

    # forward pass for model
    def forward(self, x):
        return self.model(x)

In [ ]:
def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(loader):
        # want to use gpu
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = model.loss_criterion(output, target)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        if batch_idx % 100 == 0:
            print(f'Batch: {batch_idx}/{len(loader)} Loss: {loss.item():.4f}')

    return total_loss / len(loader), 100. * correct / total

In [ ]:
def validate(model, loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = model.loss_criterion(output, target)

            total_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    return total_loss / len(loader), 100. * correct / total

In [ ]:
def check_image_paths(df, num_samples=5):
    sample_ids = df['id'].head(num_samples)
    print("\nChecking image paths:")

    base_paths = [
        '/content/drive/MyDrive/ML Project Team/dataset/images',
        '/content/drive/.shortcut-targets-by-id/1BrnEUmLFVaEPyjyPpQRW6h7Hy0kqAwAC/ML Project Team/dataset/images',
        'images'
    ]

    for path in base_paths:
        print(f"\nTrying base path: {path}")
        for img_id in sample_ids:
            full_path = os.path.join(path, f'{img_id}.jpg')
            exists = os.path.exists(full_path)
            print(f"ID {img_id}: {full_path} - {'Exists' if exists else 'Not found'}")


In [ ]:
def main():
    # device as gpu
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # filter out what we do not want (no accessories)
    df = pd.read_csv('styles.csv', on_bad_lines='skip')
    df = df[df['masterCategory'].isin(['Apparel', 'Footwear'])]

    print(f"\nInitial dataset size: {len(df)}")
    print("\nMaster category distribution:")
    print(df['masterCategory'].value_counts())

    valid_ids = []
    for idx, row in df.iterrows():
        if os.path.exists(f'images/{row["id"]}.jpg'):
            valid_ids.append(idx)

    df = df.loc[valid_ids]

    # get rid of any samples that have too little count
    article_counts = df['articleType'].value_counts()
    valid_articles = article_counts[article_counts >= 30].index
    df = df[df['articleType'].isin(valid_articles)]

    print(f"\nDataset size after filtering: {len(df)}")
    print("\nArticle type distribution:")
    print(df['articleType'].value_counts())
    print("\nNumber of unique article types:", len(df['articleType'].unique()))

    # train/test split 0.2
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42,
                                      stratify=df['articleType'])

    # Define transforms
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # make dataset from the splits
    train_dataset = FashionDataset('images', train_df, train_transform)
    val_dataset = FashionDataset('images', val_df, val_transform)

    print(f"\nNumber of classes (article types): {len(train_dataset.classes)}")
    print(f"Article types: {train_dataset.classes}")
    print(f"Number of training images: {len(train_dataset)}")
    print(f"Number of validation images: {len(val_dataset)}")

    train_loader = DataLoader(
        train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    # Initialize model
    model = FashionResNet(len(train_dataset.classes)).to(device)
    scaler = torch.cuda.amp.GradScaler()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',
                                                         factor=0.5, patience=3)

    # Training loop
    num_epochs = 30
    best_val_acc = 0
    # Increase patience counter by 1 for each epoch that validation accuracy does
    # not increase, after 10 epochs, stop training early.
    patience = 10
    patience_counter = 0

    for epoch in range(num_epochs):
        # train
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # tqdm for progress bars
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = model.loss_criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100. * correct / total

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc='Validation'):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = model.loss_criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100. * correct / total

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

        scheduler.step(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_acc': epoch_acc,
                'val_acc': val_acc,
                'classes': train_dataset.classes,
                'class_to_idx': train_dataset.class_to_idx
            }, 'best_fashion_model.pth')
            print(f'New best model saved with validation accuracy: {val_acc:.2f}%')
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f'Early stopping triggered after {epoch+1} epochs')
            break

    print(f'\nTraining completed. Best validation accuracy: {best_val_acc:.2f}%')

In [ ]:
if __name__ == '__main__':
    main()

Using device: cuda

Initial dataset size: 30616

Master category distribution:
masterCategory
Apparel     21397
Footwear     9219
Name: count, dtype: int64

Dataset size after filtering: 12404

Article type distribution:
articleType
Tshirts            2906
Shirts             1815
Casual Shoes       1191
Kurtas              870
Sports Shoes        721
Tops                576
Heels               392
Flip Flops          362
Sandals             352
Briefs              333
Trousers            317
Sweatshirts         271
Sweaters            263
Formal Shoes        261
Jeans               257
Shorts              212
Flats               179
Jackets             168
Innerwear Vests     149
Track Pants         145
Kurtis              115
Bra                 107
Trunk                80
Dresses              73
Leggings             61
Kurta Sets           56
Capris               55
Suspenders           40
Tunics               40
Skirts               37
Name: count, dtype: int64

Number of unique art

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-43-4572c153c9a5>:82: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



Total parameters: 11,191,902
Trainable parameters: 15,390
Frozen parameters: 11,176,512


Epoch 1/30:   0%|          | 0/156 [00:00<?, ?it/s]<ipython-input-43-4572c153c9a5>:105: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validation: 100%|██████████| 39/39 [02:31<00:00,  3.87s/it]


Epoch 1/30:
Train Loss: 1.7668, Train Acc: 51.58%
Val Loss: 1.1978, Val Acc: 64.01%
New best model saved with validation accuracy: 64.01%


...

Epoch 28/30:
Train Loss: 0.5651, Train Acc: 80.92%
Val Loss: 0.6430, Val Acc: 78.15%
New best model saved with validation accuracy: 78.15%


Validation: 100%|██████████| 39/39 [00:11<00:00,  3.44it/s]


Epoch 29/30:
Train Loss: 0.5678, Train Acc: 80.85%
Val Loss: 0.6361, Val Acc: 78.27%
New best model saved with validation accuracy: 78.27%


Validation: 100%|██████████| 39/39 [00:10<00:00,  3.77it/s]

Epoch 30/30:
Train Loss: 0.5677, Train Acc: 81.26%
Val Loss: 0.6521, Val Acc: 77.75%

Training completed. Best validation accuracy: 78.27%


In [ ]:
import os
import torch
import json

# save model in drive
save_path = '/content/drive/MyDrive/ML Project Team/saved_models/'
os.makedirs(save_path, exist_ok=True)

temp_model = torch.load('best_fashion_model.pth')

# new name set for model
permanent_save_path = f'{save_path}expanded_final_model.pth'
torch.save(temp_model, permanent_save_path)

print(f"Model saved to: {permanent_save_path}")
print("Saved model contents:")
for key in temp_model.keys():
    print(f"- {key}")

<ipython-input-45-50e36f5b2c7b>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  temp_model = torch.load('best_fashion_model.pth')


Model saved to: /content/drive/MyDrive/ML Project Team/saved_models/expanded_final_model.pth
Saved model contents:
- epoch
- model_state_dict
- optimizer_state_dict
- train_acc
- val_acc
- classes
- class_to_idx


In [ ]:
print(train)

NameError: name 'self' is not defined